In [1]:
import os, sys, time, resource, re, gc, shutil
from multiprocess import Pool
from functools import partial
from tabulate import tabulate
from urllib.parse import urlparse, parse_qsl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mongoengine.queryset.visitor import Q
import django
from django.db.models import Count, Sum
sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from django.forms.models import model_to_dict

from scoping.models import *
from tmv_app.models import *
from cities.models import *
from scipy import stats
import ipy_table as tbl

run_id=359
qid = 1603

# Produce df with title, author, abstract, doi

In [2]:
#import spacy
from geotext import GeoText
docs = Doc.objects.filter(query=qid,cities__isnull=True)
print(docs.count())

docplaces = 0

spacy = False
geotext = True

for d in docs:
    if spacy:
        nlp = spacy.load('en')
        doc = nlp(u'London is a big city in the United Kingdom.')
        doc = nlp(d.content)
        #print(d.content)
        places = [x for x in doc.ents if x.label_=="GPE"]
        if len(places) > 0:
            docplaces+=1
            print("############### \n## SPACY")
            for place in places:
                print(place.label_, place.text)
                try:
                    dbc = City.objects.filter(
                        name__unaccent=place.text
                    ).order_by('-population').first().name_std
                except:
                    dbc = "not found"
                print(dbc)
 
    if geotext:
        gplaces = set(GeoText(d.content).cities)
        if len(gplaces) > 0:
            for place in gplaces:
                try:
                    dbc = City.objects.filter(
                        alt_names__name__unaccent=place
                    ).order_by('-population').first()
                    d.cities.add(dbc)
                    docplaces+=1
                except:
                    dbc = "not found"
        
print(docplaces)

6445
0


In [2]:
badcities = ['Metro','Most','Sim','Young','University','Green','Much','Mobile','Federal','Along','Of','Laplace']
cities = City.objects.filter(doc__query=qid).exclude(name__in=badcities)
cities = cities.annotate(
    n = Count('doc')
).order_by('-n').values('name','country__name','n','location')

docs = Doc.objects.filter(query=qid,cities__isnull=False).exclude(cities__name__in=badcities)
print(docs.count())


3307


In [3]:
df = pd.DataFrame.from_dict(list(cities))
df.to_csv("../data/place_mentions.csv")
df.head(135)

,country__name,location,n,name
0,China,"[116.39723, 39.9075]",155,Beijing
1,United Kingdom,"[-0.12574, 51.50853]",135,London
2,United States,"[-74.00597, 40.71427]",119,New York City
3,Sweden,"[18.0649, 59.33258]",80,Stockholm
4,China,"[121.45806, 31.22222]",71,Shanghai
5,Australia,"[144.96332, -37.814]",49,Melbourne
6,Australia,"[151.20732, -33.86785]",48,Sydney
7,United States,"[-122.67621, 45.52345]",43,Portland
8,Canada,"[-79.4163, 43.70011]",43,Toronto
9,Chile,"[-70.64827, -33.45694]",41,Santiago


In [14]:
d_dict = []
for d in docs.distinct():
    d_dict.append({
        'UT': d.UT.UT,
        'title': d.title,
        'abstract': d.content,
        'authors': d.authors,
        'year': d.PY,
        'doi': d.wosarticle.di,
        'cities': '; '.join([x.name for x in d.cities.all()])
    })

df = pd.DataFrame.from_dict(d_dict)
df.head(10)



,UT,abstract,authors,cities,doi,title,year
0,WOS:000312388100001,As the major source of greenhouse gas (GHG) em...,"Song, D, Su, MR, Yang, J, Chen, B",Beijing,10.1100/2012/613721,Greenhouse Gas Emission Accounting and Managem...,2012
1,WOS:000358458100039,"Today, buildings are responsible for more than...","Atmaca, A, Atmaca, N",Gaziantep,10.1016/j.enbuild.2015.06.008,Life cycle energy (LCEA) and carbon dioxide em...,2015
2,WOS:000312696400046,Cities worldwide are increasingly becoming age...,"Creutzig, F, Muhlhoff, R, Romer, J",Barcelona; Freiburg; Sofia,10.1088/1748-9326/7/4/044042,Decarbonizing urban transport in European citi...,2012
3,WOS:000189107200002,"Chicago's large, diverse automotive industry s...","Lewis, R",Chicago,None,Local production practices and Chicago's autom...,2003
4,WOS:000086382000003,By North American standards Toronto is a conce...,"Filion, P",Toronto,10.1068/c2m,Balancing concentration and dispersion? Public...,2000
5,WOS:000330317400003,The policy actions of senior levels of governm...,"Nelles, J",Detroit,10.1177/1078087412458255,Regionalism Redux: Exploring the Impact of Fed...,2013
6,WOS:000246315600010,"The building sector, one of the fastest growin...","Rey, FJ, Velasco, E, Varela, F",Kyoto; Brussels,10.1016/j.enbuild.2006.07.009,Building Energy Analysis (BEA): A methodology ...,2007
7,WOS:000279133900007,"Between the World Wars, public transportation ...","McCammack, B",Chicago; New York City,None,"""MY GOD, THEY MUST HAVE RIOTS ON THOSE THINGS ...",2010
8,WOS:000393803400003,"Vienna, Austria reduced the car share of trips...","Buehler, R, Pucher, J, Altshuler, A",Vienna,10.1080/15568318.2016.1251997,Vienna's path to sustainable transport,2017
9,WOS:000391900000021,Urban mobility impacts urban life to a great e...,"Gal, A, Mandelbaum, A, Schnitzler, F, Senderov...",Dublin,10.1016/j.is.2015.12.001,Traveling time prediction in scheduled transpo...,2017


In [12]:
dids = set(list(docs.values_list('id',flat=True)))

dts = DocTopic.objects.filter(run_id=run_id,doc__id__in=dids)

dtdf = pd.DataFrame.from_dict(
    list(dts.values('doc__UT__UT','topic__title','score'))
).rename(index=str, columns={"doc__UT__UT": "UT"}).pivot('UT','topic__title','score').fillna(0).reset_index()

dtdf.head()

topic__title,UT,Active travel,Air quality,Building energy standards,Building water use,Bus rapid transit,Car parking,Charging stations,Climate change mitigation,Congestion charging,...,Rail transit,Road tolls,System design,Thermal comfort,Transport services & provisioning,Travel behaviour,Urban form,Waste management,Waste water,e-Vehicles
0,WOS:000071306300001,0.008273,0.000000,0.019368,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.002878,0.033369,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,WOS:000071344100003,0.029842,0.000000,0.000000,0.0,0.0,0.0,0.0,0.002607,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,WOS:000071367300003,0.000000,0.001183,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.009205,0.082836,0.005228,0.000000
3,WOS:000072032700013,0.027916,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.006881,...,0.0,0.000000,0.002238,0.000000,0.0,0.067567,0.026691,0.000000,0.000000,0.003944
4,WOS:000072238900006,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.008941,...,0.0,0.002622,0.004704,0.000000,0.0,0.000000,0.011385,0.002546,0.000000,0.000000


In [15]:
mdf = pd.merge(df,dtdf)

mdf.head()

,UT,abstract,authors,cities,doi,title,year,Active travel,Air quality,Building energy standards,...,Rail transit,Road tolls,System design,Thermal comfort,Transport services & provisioning,Travel behaviour,Urban form,Waste management,Waste water,e-Vehicles
0,WOS:000312388100001,As the major source of greenhouse gas (GHG) em...,"Song, D, Su, MR, Yang, J, Chen, B",Beijing,10.1100/2012/613721,Greenhouse Gas Emission Accounting and Managem...,2012,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.021141,0.0,0.000000,0.001425
1,WOS:000358458100039,"Today, buildings are responsible for more than...","Atmaca, A, Atmaca, N",Gaziantep,10.1016/j.enbuild.2015.06.008,Life cycle energy (LCEA) and carbon dioxide em...,2015,0.000000,0.002836,0.084028,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.011784,0.0,0.000000,0.000000
2,WOS:000312696400046,Cities worldwide are increasingly becoming age...,"Creutzig, F, Muhlhoff, R, Romer, J",Barcelona; Freiburg; Sofia,10.1088/1748-9326/7/4/044042,Decarbonizing urban transport in European citi...,2012,0.014342,0.031274,0.000000,...,0.000000,0.0,0.00000,0.0,0.000000,0.0,0.019777,0.0,0.000000,0.001581
3,WOS:000189107200002,"Chicago's large, diverse automotive industry s...","Lewis, R",Chicago,None,Local production practices and Chicago's autom...,2003,0.000000,0.000000,0.000000,...,0.000000,0.0,0.00000,0.0,0.005714,0.0,0.000000,0.0,0.004707,0.000000
4,WOS:000086382000003,By North American standards Toronto is a conce...,"Filion, P",Toronto,10.1068/c2m,Balancing concentration and dispersion? Public...,2000,0.008338,0.012156,0.000000,...,0.008114,0.0,0.00114,0.0,0.000000,0.0,0.018878,0.0,0.007548,0.000000


In [16]:
df.to_csv('/media/W/tmp/cities_places.csv')

mdf.to_csv('/media/W/tmp/cities_places_topics.csv')